# Biomaker CA: performing advanced runs on a configuration

In this colab we show how to run models on a configuration and how to evaluate them.

This colab allows to choose whether to perform sexual and/or asexual reproduction.
It also allows for sparse computations of agent logics.

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Imports

In [ ]:
#@title install selforg package
# install the package locally
!pip install --upgrade -e git+https://github.com/google-research/self-organising-systems.git#egg=self_organising_systems&subdirectory=biomakerca
# activate the locally installed package (otherwise a runtime restart is required)
import pkg_resources
import importlib
# Reload the resources because we uninstalled and reinstalled some packages.
importlib.reload(pkg_resources)
pkg_resources.get_distribution("self_organising_systems").activate()

In [ ]:
#@title imports & notebook utilities

from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import BasicMutator
from self_organising_systems.biomakerca.mutators import RandomlyAdaptiveMutator
from self_organising_systems.biomakerca.mutators import CrossOverSexualMutator
from self_organising_systems.biomakerca.step_maker import step_env
from self_organising_systems.biomakerca.display_utils import zoom, tile2d, add_text_to_img, imshow
from self_organising_systems.biomakerca.custom_ipython_display import display
from self_organising_systems.biomakerca.env_logic import env_perform_multi_world_reproduce_update

import cv2
import numpy as np
import jax.random as jr
import jax.numpy as jp
from jax import vmap
from jax import jit
import jax
import time
import matplotlib.pyplot as plt

import tqdm
import mediapy as media
from functools import partial


def pad_text(img, text):
  font = cv2.FONT_HERSHEY_SIMPLEX
  orgin = (5, 15)
  fontScale = 0.5
  color = (0, 0, 0)
  thickness = 1

  # ensure to preserve even size (assumes the input size was even.
  new_h = img.shape[0]//15
  new_h = new_h if new_h % 2 == 0  else new_h + 1
  img = np.concatenate([np.ones([new_h, img.shape[1], img.shape[2]]), img], 0)
  img = cv2.putText(img, text, orgin, font, fontScale, color, thickness, cv2.LINE_AA)
  return img

## Select the configuration, the agent logic and the mutator

Set soil_unbalance_limit to 0 to reproduce the original environment. Set it to 1/3 for having self-balancing environments (recommended).

In [ ]:
ec_id = "persistence" #@param ['persistence', 'pestilence', 'collaboration', 'sideways']
env_width_type = "wide" #@param ['wide', 'landscape', 'square', 'petri', '10x', '20x']
soil_unbalance_limit = 1/3 #@param [0, "1/3"] {type:"raw"}

h = 72
if env_width_type == "10x":
  env_width_type = h * 10
if env_width_type == "20x":
  env_width_type = h * 20
else:
  env_width_type = evm.infer_width(h, env_width_type)

env_and_config = evm.get_env_and_config(ec_id, width_type=env_width_type, h=h)
_, config = env_and_config

st_env = evm.create_multiseed_environment(h, env_width_type, config)

config.soil_unbalance_limit = soil_unbalance_limit
reproduction_type = "asexual" #@param ['both', 'asexual', 'sexual']
does_sex_matter = True #@param ['False', 'True'] {type:"raw"}

enable_asexual_reproduction = reproduction_type != "sexual"
enable_sexual_reproduction = reproduction_type != "asexual"

sex_sensitivity = 1000 #@param [1, 10, 100, 1000] {type:"raw"}

agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal",
                              make_asexual_flowers_likely=enable_asexual_reproduction,
                              make_sexual_flowers_likely=enable_sexual_reproduction,
                              init_noise=0.001, sex_sensitivity=sex_sensitivity)

n_sparse_max = 2**13 #@param ['None', '2**13', '2**12', '2**11', '2**10'] {type:"raw"}

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))
sexual_mutator = CrossOverSexualMutator(mutator, n_frequencies=64)

exp_id = "{}_sm^{}_sens^{}_w^{}".format(reproduction_type, does_sex_matter, sex_sensitivity, env_width_type)
print(exp_id)

## Optionally, modify the config for custom configurations

In [ ]:
print("Current config:")
print('\n'.join("%s: %s" % item for item in vars(config).items()))

In [ ]:
## Examples for modifying the config
## Uncomment relevant lines or do like them.

## Regardless, to trigger the recomputation of step_env and similar,
## config needs to be a new object! So, first, we create a new copy.
import copy
config = copy.copy(config)

## Change simple isolated parameters (most of them)
# config.struct_integrity_cap = 100
# config.max_lifetime = 500
## Vectors can be modified either by writing new vectors:
# config.dissipation_per_step = jp.array([0.02, 0.02])
## Or by multiplying previous values. Note that they are immutable!
# config.dissipation_per_step = config.dissipation_per_step * 2

## agent_state_size is trickier, because it influences env_state_size.
## So you can either create a new config:
## Note that you would have to insert all values that you don't want to take
## default initializations.
# config = evm.EnvConfig(agent_state_size=4)
## Or you can just modify env_state_size as well.
## (env_state_size = agent_state_size + 4) for now.
# config.agent_state_size = 4
# config.env_state_size = config.agent_state_size + 4


## Perform a simulation

Consider modifying the code to vary the extent of the simulation and video configs.

In [ ]:
# create auxiliary frames that show interesting counters.

def make_aux_frame(n_asex, n_sex):
  img = np.ones([130 if reproduction_type == "both" else 65, 550, 3])
  yorigin = 50
  if enable_asexual_reproduction:
    img = add_text_to_img(
        img, "Asexual reproductions: {}".format(n_asex),
        origin=(20, yorigin), color="black")
    yorigin = 100
  if enable_sexual_reproduction:
    img = add_text_to_img(
        img, "Sexual reproductions:  {}".format(n_sex),
        origin=(20, yorigin), color="black")
  return img

imshow(make_aux_frame(100000, 212122))

def make_nsexes_frame(n_sexes):
  img = np.ones([65, 300, 3])
  yorigin = 50
  img = add_text_to_img(
      img, "Num sexes: {}".format(n_sexes),
      origin=(20, yorigin), color="black")
  return img

imshow(make_nsexes_frame(200))

In [ ]:

@partial(jit, static_argnames=["config", "n_max_programs"])
def get_alive_programs_mask(env, config, n_max_programs):
  aid_flat = env.agent_id_grid.flatten()
  is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
  n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=n_max_programs)

  has_alive = n_alive_per_id > 0
  return has_alive

@jit
def get_alive_and_sexes(env, programs):
  has_alive = get_alive_programs_mask(env, config, N_MAX_PROGRAMS)
  all_sexes = vmap(agent_logic.get_sex)(mutator.split_params(programs)[0])
  return has_alive, all_sexes

def get_num_sexes(env, programs):
  has_alive, all_sexes = get_alive_and_sexes(env, programs)
  has_alive = np.array(has_alive)
  all_sexes = np.array(all_sexes)

  sexes = all_sexes[has_alive]
  return len(np.unique(sexes))

def run_env(
    key, programs, env, n_steps, step_f,
    curr_asexual_repr = 0, curr_sexual_repr = 0,
    zoom_sz=12,
    steps_per_frame=2, when_to_double_speed=[100, 500, 1000, 2000, 5000]):

  fps = 20
  def make_frame(env):
    return zoom(evm.grab_image_from_env(env, config),zoom_sz)

  frame = make_frame(env)

  # remember that the metrics are per step, right now, and that are 'inbetween'
  # frames, at best.
  n_asexual_repr_log = []
  n_sexual_repr_log = []
  n_sexes_log = []

  aux_frames = [make_aux_frame(0, 0)]
  num_sexes_frames = [make_nsexes_frame(get_num_sexes(env, programs))]

  out_file = "video.mp4"
  with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps, crf=18
                        ) as video:
    video.add_image(frame)
    for i in tqdm.trange(n_steps):
      if i in when_to_double_speed:
        steps_per_frame *= 2

      key, ku = jr.split(key)
      (env, programs), metrics = step_f(ku, env, programs=programs)

      if enable_asexual_reproduction:
        step_asex_repr = int(metrics["asexual_reproduction"][0].sum())
        n_asexual_repr_log.append(step_asex_repr)
        curr_asexual_repr += step_asex_repr
      if enable_sexual_reproduction:
        step_sexual_repr = int(metrics["sexual_reproduction"][0].sum())
        n_sexual_repr_log.append(step_sexual_repr)
        curr_sexual_repr += step_sexual_repr

      # get sexes alive
      num_sexes = get_num_sexes(env, programs)
      n_sexes_log.append(num_sexes)
      if i % steps_per_frame == 0:
        video.add_image(make_frame(env))
        aux_frames.append(make_aux_frame(curr_asexual_repr, curr_sexual_repr))
        num_sexes_frames.append(make_nsexes_frame(num_sexes))


  media.show_video(media.read_video(out_file))
  media.show_video(aux_frames, fps=fps)
  media.show_video(num_sexes_frames, fps=fps)
  ret_metrics = {'n_asexual_repr_log': n_asexual_repr_log,
                 'n_sexual_repr_log': n_sexual_repr_log,
                 'n_sexes_log': n_sexes_log,
                 'curr_asexual_repr': curr_asexual_repr,
                 'curr_sexual_repr': curr_sexual_repr}
  return programs, env, ret_metrics

In [ ]:
key = jr.PRNGKey(42)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 512

# for 20x environments, you need shorter videos.
n_steps = 15000

# on what FRAME to double speed.
when_to_double_speed = [100, 200, 300, 400, 500]
# on what FRAME to reset speed.
when_to_reset_speed = []
fps = 20
# this affects the size of the image. If this number is not even, the resulting
# video *may* not be supported by all renderers.
zoom_sz = 4

ku, key = jr.split(key)
programs = vmap(agent_logic.initialize)(jr.split(ku, N_MAX_PROGRAMS))
ku, key = jr.split(key)
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env

step_f = partial(step_env, config=config, agent_logic=agent_logic, do_reproduction=True,
          enable_asexual_reproduction=enable_asexual_reproduction,
          enable_sexual_reproduction=enable_sexual_reproduction,
          does_sex_matter=does_sex_matter,
          mutate_programs=True, mutator=mutator, sexual_mutator=sexual_mutator,
          n_sparse_max=n_sparse_max, return_metrics=True)

step = 0
# how many steps per frame we start with. This gets usually doubled many times
# during the simulation.
# In the article, we usually use 2 or 4 as the starting value, sometimes 1.
steps_per_frame = 2

ku, key = jr.split(key)
programs, env, metrics = run_env(
    ku, programs, env, n_steps, step_f, zoom_sz=6)



def running_average(a, n):
  a = np.concatenate([np.full([n], a[0]), a], axis=0)
  return np.convolve(a, np.ones(n)/n, mode="valid")

if enable_asexual_reproduction:
  plt.plot(running_average(metrics['n_asexual_repr_log'], 100), label="n_asexual_repr_log")
if enable_sexual_reproduction:
  plt.plot(running_average(metrics['n_sexual_repr_log'], 100), label="n_sexual_repr_log")
plt.legend()
plt.show()

plt.plot(running_average(metrics['n_sexes_log'], 100), label="n_sexes_log")
plt.legend()
plt.show()

aid_flat = env.agent_id_grid.flatten()
is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=N_MAX_PROGRAMS)
alive_programs = programs[n_alive_per_id>0]
print("Extracted {} programs.".format(alive_programs.shape[0]))
print("sexes:", vmap(agent_logic.get_sex)(mutator.split_params(alive_programs)[0]))

In [ ]:
# continue...
n_steps = 25000
ku, key = jr.split(key)

programs, env, metrics = run_env(
    ku, programs, env, n_steps, step_f, zoom_sz=6,
    steps_per_frame=64, when_to_double_speed=[],
    curr_asexual_repr=metrics['curr_asexual_repr'],
    curr_sexual_repr=metrics['curr_sexual_repr'])


if enable_asexual_reproduction:
  plt.plot(running_average(metrics['n_asexual_repr_log'], 100), label="n_asexual_repr_log")
if enable_sexual_reproduction:
  plt.plot(running_average(metrics['n_sexual_repr_log'], 100), label="n_sexual_repr_log")
plt.legend()
plt.show()

plt.plot(running_average(metrics['n_sexes_log'], 100), label="n_sexes_log")
plt.legend()
plt.show()

aid_flat = env.agent_id_grid.flatten()
is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=N_MAX_PROGRAMS)
alive_programs = programs[n_alive_per_id>0]
print("Extracted {} programs.".format(alive_programs.shape[0]))
print("sexes:", vmap(agent_logic.get_sex)(mutator.split_params(alive_programs)[0]))
